In [1]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input, Merge, Dropout
import numpy as np
import pandas as pd
import theano
import theano.tensor as T
import numpy.ma as ma
import pickle
from sklearn.cross_validation import train_test_split

Using Theano backend.
Using gpu device 0: GRID K520 (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5005)


In [2]:
def loss_func(y_true, y_pred):
    temp = np.dot(y_true, y_pred)
    temp_true = y_true.nonzero_values()
    temp_pred = temp.nonzero_values()
    temp_pred = temp_pred / temp_true
    loss = T.mean((temp_pred - temp_true)**2)
    return loss

In [3]:
X = pickle.load(open('data/X.pkl', 'rb'))
y = pickle.load(open('data/y.pkl', 'rb'))

In [4]:
user_data = pickle.load(open('data/user_matrix.pkl', 'rb'))

In [5]:
X_train, X_test, y_train, y_test, user_train, user_test = train_test_split(X, y, user_data)

#### Sanity Checks

In [6]:
print(X_train.shape)
print(y_train.shape)
print(user_train.shape)
print(user_test.shape)
print(X_test.shape)
print(y_test.shape)

(334063, 843)
(334063, 843)
(334063, 25)
(111355, 25)
(111355, 843)
(111355, 843)


In [11]:
y.shape

(445418, 843)

In [7]:
user_dim = user_data.shape[1]
rating_dim = X.shape[1]
user_encoding_dim = 10
encoding_dim = 64

### Free up memory

In [8]:
del X
del y
del user_data

In [9]:
 # this is our input placeholder
from keras.layers.normalization import BatchNormalization
from numpy.random import seed
branch1 = Sequential()
branch1.add(Dense(user_encoding_dim, input_shape = (user_dim,), init = 'normal', activation = 'relu'))
branch1.add(Dropout(0.2))
 
branch2 = Sequential()
branch2.add(Dense(encoding_dim, input_shape =(rating_dim,), init = 'normal', activation = 'relu'))
branch1.add(BatchNormalization())
branch2.add(Dropout(0.2))
 
model = Sequential()
model.add(Merge([branch1, branch2], mode = 'concat'))
model.add(Dense(rating_dim, init = 'normal', activation = 'sigmoid'))
model.compile(loss = loss_func, optimizer = 'adadelta')
seed(2017)

In [10]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 10)            260                                          
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 10)            0                                            
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 10)            40                                           
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 64)            54016                                        
___________________________________________________________________________________________

In [11]:
losses = []
test_scores = []

In [12]:
def get_predictions(test, model, X_1, X_2):
    y_pred = np.zeros(test.shape[0])
    for i, t in enumerate(test):
        pred = model.predict([X_1[t[2]].reshape(1, -1), X_2[t[2]].reshape(1, -1)])
        pred = pred[0][t[2]]
        y_pred[i] = pred * 5
    return y_pred

In [27]:
for i in range(20):
    history = model.fit([user_train, X_train], y_train, nb_epoch=5, batch_size = 100)
    losses.extend(history.history['loss'])
    test_score = model.evaluate([user_test, X_test], y_test)
    test_scores.append(test_score)
    print('\ntraining error: {}'.format(test_score))
    if test_score > test_scores[-2]:
        break

Epoch 1/5
334063/334063 [==============================] - 19s - loss: 0.0215    
Epoch 2/5
334063/334063 [==============================] - 19s - loss: 0.0214    
Epoch 3/5
334063/334063 [==============================] - 19s - loss: 0.0213    
Epoch 4/5
334063/334063 [==============================] - 19s - loss: 0.0212    
Epoch 5/5
111200/111355 [============================>.] - ETA: 0s
training error: 0.021719344524941578
Epoch 1/5
334063/334063 [==============================] - 19s - loss: 0.0211    
Epoch 2/5
334063/334063 [==============================] - 19s - loss: 0.0211    
Epoch 3/5
334063/334063 [==============================] - 19s - loss: 0.0209    
Epoch 4/5
334063/334063 [==============================] - 19s - loss: 0.0209    
Epoch 5/5
111168/111355 [============================>.] - ETA: 0s
training error: 0.021505777437117277
Epoch 1/5
334063/334063 [==============================] - 19s - loss: 0.0208    
Epoch 2/5
334063/334063 [=============================

## Save progress

In [68]:
pickle.dump(losses, open('data/losses.pkl', 'wb'))
pickle.dump(test_scores, open('data/test_mse.pkl', 'wb'))
model.save('model.h5')
model.save_weights('model_weights.h5')